In [1]:
import os
%pwd

'd:\\Text_Summarizer\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Text_Summarizer'

In [3]:
import torch

# Check if GPU is available
print("CUDA Available:", torch.cuda.is_available())

# Print your GPU name (if available)
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("No GPU found.")


CUDA Available: False
No GPU found.


In [3]:
from dataclasses import dataclass
from pathlib import Path
import transformers
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_from_disk
import torch



d:\Text_Summarizer\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path 
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    do_eval: bool
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int



In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories


from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories
from src.textSummarizer.entity import ModelTrainerConfig


class ConfigurationManager:
    def __init__(self,
                 config_path: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH):
        # Load YAML files
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        # Ensure root directory exists
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        # Extract config and parameters
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        # Ensure model trainer root directory exists
        create_directories([config.root_dir])

        # Construct and return config dataclass
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            do_eval=params.do_eval,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        return model_trainer_config


In [7]:
from transformers import PegasusForConditionalGeneration,PegasusTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
from datasets import load_from_disk

In [8]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk
import os
import logging

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        self.logger = logging.getLogger(__name__)

    def train(self):
        # Device setup
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.logger.info(f"Using device: {device}")

        # Load dataset
        try:
            dataset = load_from_disk(self.config.data_path)
            self.logger.info("Dataset loaded successfully.")
        except Exception as e:
            self.logger.error(f"Failed to load dataset: {e}")
            raise

        # Load tokenizer and model
        try:
            tokenizer = PegasusTokenizer.from_pretrained(self.config.model_ckpt)
            model = PegasusForConditionalGeneration.from_pretrained(self.config.model_ckpt).to(device)
            self.logger.info("Model and tokenizer loaded successfully.")
        except Exception as e:
            self.logger.error(f"Failed to load model or tokenizer: {e}")
            raise

        # Data collator
        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        # Training arguments
        training_args = Seq2SeqTrainingArguments(
            output_dir=str(self.config.root_dir),
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            save_steps=self.config.save_steps,
            eval_steps=self.config.eval_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            predict_with_generate=True,
            do_eval=self.config.do_eval,
            logging_dir=os.path.join(str(self.config.root_dir), "logs")
        )

        # Trainer
        trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            tokenizer=tokenizer,
            data_collator=data_collator,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"] if self.config.do_eval else None
        )

        try:
            self.logger.info("Starting training...")
            trainer.train()
            self.logger.info("Training completed.")
        except Exception as e:
            self.logger.error(f"Error during training: {e}")
            raise

        # Save model and tokenizer
        model_save_path = os.path.join(str(self.config.root_dir), "model")
        tokenizer_save_path = os.path.join(str(self.config.root_dir), "tokenizer")
        model.save_pretrained(model_save_path)
        tokenizer.save_pretrained(tokenizer_save_path)
        self.logger.info(f"Model and tokenizer saved to {model_save_path} and {tokenizer_save_path}.")

In [ ]:
import torch
print(torch.cuda.device_count())


In [9]:
config=ConfigurationManager()
model_trainer_config=config.get_model_trainer_config()
model_trainer =ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-05-23 10:12:07,714: INFO: common: YAML file loaded successfully: config\config.yaml]
[2025-05-23 10:12:08,132: INFO: common: YAML file loaded successfully: params.yaml]
[2025-05-23 10:12:08,140: INFO: common: Created directory at: artifacts]
[2025-05-23 10:12:08,150: INFO: common: Created directory at: artifacts/model_trainer]
[2025-05-23 10:12:08,150: INFO: 124806720: Using device: cpu]
[2025-05-23 10:12:12,587: INFO: 124806720: Dataset loaded successfully.]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[2025-05-23 10:14:45,398: INFO: 124806720: Model and tokenizer loaded successfully.]


C:\Users\ELCOT\AppData\Local\Temp\ipykernel_8524\124806720.py:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


[2025-05-23 10:15:29,838: INFO: 124806720: Starting training...]


: 